# This ____ does not exist

This is a configurable notebook for making a DCGAN on whatever you like!

Point to a folder contating raw images of something you want to learn to generate images of, tweak some settings and go!

face_recognition yields poor and slow results so this uses an opencv and mediapipe solution I made. If you plan on using it you will need opencv and mediapipe...

Uncomment and run the following code to install it.

In [ ]:
#!pip install opencv-python
#!pip install mediapipe

### Edit the variables below then run the cell.
Then you can scroll past it and run the training and visualization cell.

In [ ]:
from PIL import Image, ImageOps
import numpy as np
import glob
# Comment this out if not using my face recognition
from faceRecognitionModule import FaceDetector
# Comment the google colab imports out if not using google colab.
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
from tensorflow import keras
import tensorflow as tf
from keras import layers

########################################{ CLEANING AND STORAGE SETTINGS }########################################
# Change these how you wish.
raw_path = 'drive/MyDrive/raw_imgs'
clean_path = 'drive/MyDrive/clean_imgs'
npy_path = 'drive/MyDrive/GAN_npy'
gen_path = 'drive/MyDrive/generator'

# Size of generated and resized images. MUST BE FACTOR OF 8!
# Default is 64,64. If you go much larger you will want to change GAN architecture!
# Keep 1:1 ratio!
img_size = (64,64)
img_shape = (img_size[0],img_size[1],3) # leave this alone

# Save .npy array to directory?
save_npy = True

# Use face detection to crop images to face?
crop_to_face = True

# simple crop and scale to target size (center crop to 1:1 ratio then scales down. Be mindful that important features don't get cut off)
simple_crop = False

# Simply resize the images (be mindful of original resolution to avoid 'squished images')
simple_resize = False
### ONLY ONE IS USED, RESPECTIVE PRIORITY!!! if crop_to_face == True: nothing else will execute!

# Pad size on face cropping (fraction) EX: 5 = 1/5 of original size is added to cropped image.
# (only relevant if crop_to_face == True)
pad_size = 5

########################################{ GAN TRAINING SETTINGS }########################################

# Latent noise dim, default 128
latent_dim = 128

# Epochs (Tweak this depending on the size of your dataset, very small datasets will require many many epochs and very large will require not nearly as many)
epochs = 30

# Learning rate, expirement with this if you're getting many artifacts or can want to try get away with fewer epochs
learning_rate = 0.0001

########################################{ END OF HIGH LEVEL USER CONFIGURATION }#########################



# You shouldn't need to change anything else, run this cell and go to the next.


################################################################{ CLEANING }################################################################

def detect_face(image):
    im, bbox = detector.findFaces(np.array(image),draw=False)

    if len(bbox) > 0: # if face was detected
        # get crop coords
        x, y, w, h = bbox[0][0]

        # pad them
        pad = np.min([image.size[0],image.size[1]]) // pad_size
        x -= pad
        y -= pad
        x1, y1, = x+w, y+h
        x1 += pad
        y1 += pad
            
        # do some ugly padding management to avoid overshoot
        if x < 0: x = 0
        if y < 0: y = 0
        if x > image.size[0]: x = image.size[0]
        if y > image.size[1]: y = image.size[1]
        if x1 > image.size[0]: x1 = image.size[0]
        if y1 > image.size[1]: y1 = image.size[1]
        if x1 < 0: x1 = 0
        if y1 < 0: y1 = 0
            
        # simple crop
        im = im[y:y1,x:x1]

        # anti alias scale to desired ratio
        im = Image.fromarray(im)
        im = im.resize(img_size,Image.ANTIALIAS)
            
        im.save(f"{clean_path}/img_{i}.jpg")
            
        print(f'processed image {i}', end='\r')
    else: # if face is not detected we skip
        print(f"Rejected image {i}", end='\r')

def simple_resizer(im):
    im = im.resize(img_size,Image.ANTIALIAS)
            
    im.save(f"{clean_path}/img_{i}.jpg")
    print(f'processed image {i}', end='\r')

def simple_cropper(img):
    width, height = img.size
    # 1:1 crop calc
    if width > height:
        left = (width - height) / 2
        right = left + height
        top = 0
        bottom = height
    elif width < height:
        top = (height - width) / 2
        bottom = top + width
        left = 0
        right = width

    img = img.crop((left, top, right, bottom))
    img = img.resize(img_size, Image.ANTIALIAS)
    img.save(f"{clean_path}/img_{i}.jpg")
    print(f'processed image {i}', end='\r')

# Avoid error if we're not using it.
if crop_to_face: detector = FaceDetector()

# process images
for i,img in enumerate(glob.glob(raw_path+'*')):
    image = Image.open(img)
    image = ImageOps.exif_transpose(image)
    if crop_to_face: detect_face(image)
    elif simple_resize: simple_resizer(image)
    elif simple_crop: simple_cropper(image)


# add cleaned images to np.array to feed to our GAN.
image_array = np.zeros((len(glob.glob(clean_path+'/*')), img_size[0], img_size[1], 3), dtype=np.float32)
for i, image_path in enumerate(glob.glob(clean_path+'/*')):
    with Image.open(image_path) as img:
        img = np.array(img)
        image_array[i] = (img / 127.5) - 1.0
        print(f'Loaded image #{i+1}', end='\r')

if save_npy: np.save(npy_path+'/imgs.npy',image_array)

################################################################{ CLEANING END }################################################################





################################################################{ GAN BEGIN }###################################################################


# modified Keras DCGAN
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense((img_size[0] // 8) * (img_size[0] // 8) * 128),
        layers.Reshape(((img_size[0] // 8), (img_size[0] // 8), 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="tanh"),
    ],
    name="generator",
)

discriminator = keras.Sequential(
    [
        keras.Input(shape=img_shape),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)

class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    g_optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

### Where the magic happens

once that finishes running you can run the cell below to train it.

In [97]:
gan.fit(
    image_array, epochs=epochs
)#                        ^  swap this out with another number if you want to do more or less epochs...

And run this cell to see what sort of images it's generating.

You'll need to expirement around a bit to see how many epochs to do for the size of your dataset.

In [ ]:
random_latent_vectors = tf.random.normal(shape=(1, gan.latent_dim))
generated_images = gan.generator(random_latent_vectors)
import matplotlib.pyplot as plt
# ugly one liner that goes from [-1,1] eagle tensor float to [0,255] numpy int
plt.imshow(((np.array(generated_images[0])+1)*127.5).astype(np.uint8))

Save the generator if you like

In [ ]:
gan.generator.save(gen_path+'/generator.h5')